In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [ ]:
def sheardata(sav):
    data = np.load(sav)
    # print(data.files)
    Ly = 2.*np.pi
    nsav = data['nhst']
    nx = data['nx']; dy = data['dx']
    D = data['D']
    w = data['w']; v = data['v']
    m = data['m']; alpha = data['eta']
    x = np.linspace(0,Ly,nx)
    X, Y = np.meshgrid(x,x)
    navg = np.sum(nsav[-1,:,:]*dy,axis=0)/Ly
    nmax = np.max(navg)
    return nsav, navg, nmax, w, v, m, alpha, x, X, Y

In [ ]:
f = sorted(glob.glob('./inoutflux_savez/pst-pe-0030-m-4-*.npz'))
fig, ax = plt.subplots()
for i,j in enumerate(f):
    nsav, navg, nmax, w, v, m, alpha, x, X, Y = sheardata(j)
    # print(j,nmax)
    navg_x = -np.gradient(navg)
    ax.plot(x/(2*np.pi),np.abs(navg_x)/np.max(navg_x),label=r'$\alpha =$'+str(alpha))
# ax.set_ylim(-0.5,1.5)
ax.set_xlim(0,1)
ax.set_title(r'$\psi_{shear}(m =$'+str(m)+r'$)$',fontsize=16)
ax.set_xlabel(r'$x$',fontsize=16)
ax.set_ylabel(r'$\langle n \rangle_y$',fontsize=18)
# ax.legend(loc='best',shadow=True)
plt.grid()
plt.tight_layout()
# savefile = './plots/navgshear-m-01.pdf'
# plt.savefig(savefile,bbox_inches='tight')
plt.show()

In [ ]:
m_val = [1,2,3,4]; tol = 1.5e-1; stor_val = {}
for q in range(len(m_val)):
    pth = './inoutflux_savez/pst-pe-0030-m-'+str(m_val[q])+'-*.npz'
    stor_val[q] = [[],[],[]]
    f = sorted(glob.glob(pth))
    for m,n in enumerate(f):
        nsav, navg, nmax, w, v, m, alpha, x, X, Y = sheardata(n)
        navg_x = -np.gradient(navg)
        navg_x_norm = np.abs(navg_x/np.max(navg_x))
        stair_count = 0
        stair_length = 0
        length_arr = []
        for i in range(len(navg_x_norm)):
            if i == len(navg_x_norm)-1 and navg_x_norm[i-1] <= tol:
                stair_count += 1
                stair_length += 1
                length_arr.append(stair_length)
            elif navg_x_norm[i] <= tol:
                stair_length += 1
            else:
                if navg_x_norm[i-1] <= tol:
                    # print(i)
                    stair_count += 1
                    length_arr.append(stair_length)
                    stair_length = 0
                else:
                    pass
        avg_stair_length = sum(length_arr)/stair_count
        # print(avg_stair_length)
        stor_val[q][0].append(alpha)
        stor_val[q][1].append(avg_stair_length/len(navg_x_norm))
        stor_val[q][2].append(len(length_arr))

In [ ]:
fig, ax = plt.subplots()
for q in range(len(m_val)):
    ax.scatter(stor_val[q][0],stor_val[q][1],label=r'$m = $'+str(m_val[q]))
    ax.plot(stor_val[q][0],stor_val[q][1])
ax.legend(loc='best',shadow=True)
ax.set_xlabel(r'$\alpha$',fontsize=18)
ax.set_ylabel(r'$\hat{d}$',fontsize=18)
plt.grid()
plt.tight_layout()
savefile = './plots/shear-cellsize.pdf'
plt.savefig(savefile,bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots()
for q in range(len(m_val)):
    ax.scatter(stor_val[q][0],stor_val[q][2],label=r'$m = $'+str(m_val[q]))
    ax.plot(stor_val[q][0],stor_val[q][2])
ax.legend(loc='best',shadow=True)
ax.set_xlabel(r'$\alpha$',fontsize=18)
ax.set_ylabel('# steps',fontsize=18)
ax.set_ylim(0,6.5)
plt.grid()
plt.tight_layout()
savefile = './plots/shear-steps.pdf'
plt.savefig(savefile,bbox_inches='tight')
plt.show()

In [ ]:
pth = './inoutflux_savez/pst-pe-0030-m-1-*.npz'
f = sorted(glob.glob(pth)); tol = 2e-1
nsav, navg, nmax, w, v, m, alpha, x, X, Y = sheardata(f[-4])
navg_x = -np.gradient(navg)
navg_x_norm = np.abs(navg_x/np.max(navg_x))
stair_count = 0
stair_length = 0
length_arr = []
for i in range(len(navg_x_norm)):
    if navg_x_norm[i] <= tol:
        stair_length += 1
    else:
        if navg_x_norm[i-1] <= tol:
            stair_count += 1
            length_arr.append(stair_length)
            stair_length = 0
        else:
            pass
avg_stair_length = sum(length_arr)/stair_count
print(length_arr)